In [10]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import numpy as np
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time
#To do Regex
import re

# Anders Twitter keys
## Needs to be switched with Yuri's when we do the real pull

API Key: l8K8SocntIKK1kFE9rUDJfwO8

API Secret: 36cWKxQ59gWK7twlKTXRBqOMA4DIv16zO0pjlBHgfxH4szCQAG

Bearer Token: AAAAAAAAAAAAAAAAAAAAAHWGfwEAAAAATNqL7vFiLYOENaicSGgIuJbnejI%3DFNW4rnpK7FpmBDZrCfNpsdMaVKvo5HKXHedygV4r7dJSbbgsSC

# Defining the necessary functions

In [11]:
#Define Bearer token in the Windows OS Enviroment (CHANGE TOKEN HERE)
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAHWGfwEAAAAATNqL7vFiLYOENaicSGgIuJbnejI%3DFNW4rnpK7FpmBDZrCfNpsdMaVKvo5HKXHedygV4r7dJSbbgsSC'

#Defining function that will fetch the token from the enviroment
def auth():
    return os.getenv('TOKEN')

#Defining function that will pass the token for authorization and return headers for use w. the API.
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

#Defining function that creates the URL for the correct endpoint. ()
def create_url(keyword, max_results = 100):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #We only have access to recent search (7 last days)

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'max_results': max_results,
                    'next_token': {}}
    return (search_url, query_params)

# Defining function to send the “GET” request and get a "JSON" format response (if we get response code 200)

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

# Create request

In the next cell, we will set up our inputs:

    keyword = \"doge -is:retweet lang:en\"
The above specification looks for keyword "doge", exclude any retweets, and search for english tweets only bearer_token and headers from the API.

### First - define inputs

In [12]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "doge -is:retweet lang:en"


### Test field (Pulls only 10 Tweets)

In [19]:
url = create_url(keyword) #ADD THE ARGUMENT 'max_results = 100' FOR FULL 100 PULL

json_response = connect_to_endpoint(url[0], headers, url[1])

df = pd.DataFrame(json_response['data'])

df
#df['test']

Endpoint Response Code: 200


,id,text
0,1559179336141373443,$BTC $ETH $DOGE Best discord community so far...
1,1559179324976398336,$BTC $ETH $DOGE $etc Best discord community s...
2,1559179311541780483,@BitcoinMagazine @Sonofdoge_ OF DOGE NEXT BIL...
3,1559179311533264901,@cyber_bull_ RYOGE- The shiba killer of BSC. R...
4,1559179300179288071,@altcryptocom NEW GEM! 💎 @Saudishibtoken\n#Sau...
...,...,...
95,1559178540142272512,@thesavagesouls @Farewarning @opensea RYOGE- T...
96,1559178535943737345,@BTC_Archive @Sonofdoge_ OF DOGE NEXT BILION ...
97,1559178520609316864,@0xLecture @SnowLeopardsTON RYOGE- The shiba k...
98,1559178516574326790,@MonstersCoins RYOGE- The shiba killer of BSC....


# Full pull

### Define function to loop the pull several times. Saves as DataFrame.

In [14]:
def twitter_pull(maxreps=4):
    
    #Setup
    reps=0
    next_token=None
    df_result = pd.DataFrame()
    
    #Looping part
    
    for i in range(maxreps):
        #Get data from Twitter API
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        
        #Check for next_token being fetched successfully.
        #If yes, print it and continue
        if 'next_token' in json_response['meta']:
            next_token = json_response['meta']['next_token']
            print(f'next_token: {next_token}')
        #If no, print it and continue
        else:
            print('next token error')
            break
        
        df_response = pd.DataFrame(json_response['data'])
        df_result = df_result.append(df_response, ignore_index=True)
        reps+=1
        time.sleep(2)
        print(f'reps completed: {reps}')
    
    print(f'Number of tweets pulled: {len(df_result)}')
    
    return df_result
    
    

# Actual pull field:

If maxreps=4 is used, then 400 Tweets will be pulled.

In [15]:
data_df = twitter_pull(4)

Endpoint Response Code: 200
next_token: b26v89c19zqg8o3fpz5nwj2vmh1lc7l277umycofojn99
reps completed: 1
Endpoint Response Code: 200
next_token: b26v89c19zqg8o3fpz5nwj2vkyf9qnih68c2ol2rk9qwt
reps completed: 2
Endpoint Response Code: 200
next_token: b26v89c19zqg8o3fpz5nwj2vkwpul37xf3ic17bg4gvwd
reps completed: 3
Endpoint Response Code: 200
next_token: b26v89c19zqg8o3fpz5nwj2vjey1kj7np8qei4enm9it9
reps completed: 4
Number of tweets pulled: 400


In [16]:
data_df

,id,text
0,1559178481422012417,@HwanniePromotes @RichQuack RYOGE- The shiba k...
1,1559178475302293505,Your $1500 SMX pad invested instead on April 7...
2,1559178467291316226,@Ai_Fumin @freedomxbird YEAH BUT DOGE HAS A NO...
3,1559178460517617664,@Shurenn @CryptWear_TM RYOGE- The shiba killer...
4,1559178450149130247,Current #DOGE Price is $0.0766297 #Dogecoin #C...
...,...,...
395,1559175467810410497,@CryptoEmdarks @AlphGenesis\n@nystorageking\nh...
396,1559175459568599047,@Alessandro_Dste RYOGE- The shiba killer of BS...
397,1559175458666622976,$doge Top analyst price target for next week.....
398,1559175452446654465,@DeshinaBolaji @DOGECoinDrop Welcome to DOGE C...


In [18]:
uniques = data_df['id'].unique()
len(uniques)

400